# Μηχανή Ανάκτησης Πληροφορίας
### Περιγραφή
Αυτό το Jupyter notebook επιδεικνύει τη δημιουργία μιας μηχανής αναζήτησης που χρησιμοποιεί δεδομένα από την Wikipedia μέσω του API της. Οι χρήστες μπορούν να αναζητήσουν άρθρα και να εφαρμόσουν διάφορα μοντέλα αναζήτησης, όπως Boolean, Vector Space Model (VSM) και BM25, για την εύρεση των σχετικών αποτελεσμάτων.

### Αναλυτικά Βήματα:
1. Ανάκτηση άρθρων από την Wikipedia.
2. Επεξεργασία κειμένου και καθαρισμός.
3. Δημιουργία αντίστροφου ευρετηρίου.
4. Εφαρμογή μοντέλων αναζήτησης (Boolean, VSM, BM25).
5. Αξιολόγηση της απόδοσης του συστήματος αναζήτησης.

### @authors: Tilemachos Poulianas 21390304, Dionysia Giannopoulou 21390039

In [3]:
# Required Libraries
import json
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import math
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Ενδεικτικό download του nltk

In [4]:
# Download NLTK data
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tpoul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tpoul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tpoul\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Ανάκτηση Άρθρων από Wikipedia
Η συνάρτηση `fetch_wikipedia_articles` ανακτά άρθρα από την Wikipedia χρησιμοποιώντας το API της. Κάθε άρθρο που ανακτάται αποθηκεύεται σε ένα αρχείο JSON.

In [6]:
def fetch_wikipedia_articles(query, article_count):
    """
    Fetches articles from Wikipedia using a search query and a specified number of results.
    Automatically saves the articles to a JSON file named after the query.
    """
    # Base URL for Wikipedia API
    base_url = "https://en.wikipedia.org/w/api.php"
    articles = []

    # Parameters for the API request
    params = {
        'action': 'query',
        'format': 'json',
        'list': 'search',
        'srsearch': query,
        'srlimit': article_count
    }

    # Use Wikipedia API to fetch articles
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        results = response.json().get('query', {}).get('search', [])
        for result in results:
            title = result['title']
            content = fetch_article_content(title)
            if content:
                articles.append({
                    'title': title,
                    'content': preprocess_text(content)[:500],  # Limit to first 500 words
                    'url': f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"  # Create URL
                })

    # Save the fetched articles to a JSON file
    filename = f"{query.replace(' ', '_')}.json"
    if articles:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(articles, f, indent=4, ensure_ascii=False)
        print(f"Articles saved to '{filename}'.")
    else:
        print("No articles fetched.")

    return articles

def fetch_article_content(title):
    """
    Fetches the full content of a Wikipedia article given its title using the API.
    """
    # Base URL for fetching article content
    base_url = "https://en.wikipedia.org/w/api.php"
    params = {
        'action': 'query',
        'format': 'json',
        'prop': 'extracts',
        'exintro': True,
        'titles': title
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        pages = data['query']['pages']
        page = next(iter(pages.values()))
        content = page.get('extract', '')
        if content:
            content = clean_html(content)  # Clean HTML tags
        return content
    return None

## Επεξεργασία Κειμένου και Καθαρισμός HTML
Στη συνέχεια, επεξεργαζόμαστε τα άρθρα με τις συναρτήσεις `preprocess_text` και `clean_html`. Η `preprocess_text` πραγματοποιεί τοκενικοποίηση, αφαίρεση stop words και προαιρετική εφαρμογή stemming ή lemmatization.

In [8]:
def preprocess_text(raw_text, use_stemming=False, use_lemmatization=False):
    """
    Processes raw text: tokenization, stop-word removal, and optional stemming/lemmatization.
    """
    # Load stop words for filtering
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(raw_text)  # Tokenize the raw text
    cleaned_tokens = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]

    # Apply stemming if enabled
    if use_stemming:
        stemmer = PorterStemmer()
        cleaned_tokens = [stemmer.stem(token) for token in cleaned_tokens]

    # Apply lemmatization if enabled
    if use_lemmatization:
        lemmatizer = WordNetLemmatizer()
        cleaned_tokens = [lemmatizer.lemmatize(token) for token in cleaned_tokens]

    return ' '.join(cleaned_tokens)

# Clean HTML
def clean_html(raw_html):
    """
    Removes HTML tags from raw HTML content.
    """
    soup = BeautifulSoup(raw_html, "html.parser")
    return soup.get_text()

## Δημιουργία Αντίστροφου Ευρετηρίου
Η συνάρτηση `create_inverted_index` δημιουργεί ένα αντίστροφο ευρετήριο από την συλλογή των εγγράφων. Χρησιμοποιούμε το αντίστροφο ευρετήριο για να αποθηκεύσουμε τα έγγραφα ανά λέξη, προκειμένου να υποστηρίξουμε τις αναζητήσεις μας μεμονωμένα ή συνδυασμένα.

In [14]:
def create_inverted_index(docs):
    """
    Creates an inverted index from the document collection.
    """
    # Initialize an empty inverted index using defaultdict
    inverted_index = defaultdict(list)
    for doc_id, doc in enumerate(docs):
        terms = doc['content'].split()  # Split document content into terms
        for term in set(terms):
            inverted_index[term].append(doc_id)  # Map each term to the document ID
    return inverted_index

## Εφαρμογή Μοντέλων Αναζήτησης
Υλοποιούμε τα μοντέλα αναζήτησης **Boolean**, **VSM** (Vector Space Model) και **BM25** για τη λήψη των καλύτερων αποτελεσμάτων από τα άρθρα που ανακτήνηθηκαν.

In [17]:
def boolean_search(query, docs, inverted_index):
    """
    Implements Boolean retrieval model with improved handling of NOT.
    """
    query_terms = query.split()

    # Initialize an empty set for the final result
    matching_docs = set(range(len(docs)))

    # Initialize a flag to track whether the next term should be negated
    negate_next = False

    # Initialize the logical operator to None
    operator = None

    for term in query_terms:
        # Handle negation
        if term == 'NOT':
            negate_next = True
            continue

        # Check if the term is a logical operator
        if term in ['AND', 'OR']:
            operator = term
            continue  # Skip logical operators

        term_str = str(term)  # Convert term to a string

        # Perform Boolean operations (AND, OR, NOT)
        if term_str in inverted_index:
            term_results = set(inverted_index[term_str])
            if negate_next:
                matching_docs -= term_results
                negate_next = False
            else:
                # Apply logical operator
                if operator == 'AND':
                    matching_docs = matching_docs.intersection(term_results)
                elif operator == 'OR':
                    matching_docs = matching_docs.union(term_results)
                else:
                    matching_docs = term_results

    # Return the documents matching the final result set
    return [docs[doc_id] for doc_id in matching_docs]


def vsm_search(query, docs):
    """
    Implements Vector Space Model retrieval.
    """
    # Convert documents into TF-IDF vectors
    vectorizer = TfidfVectorizer()
    doc_texts = [doc['content'] for doc in docs]
    tfidf_matrix = vectorizer.fit_transform(doc_texts)

    # Convert the query into a TF-IDF vector
    query_vector = vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Rank documents based on cosine similarity
    ranked_indices = cosine_similarities.argsort()[::-1]
    ranked_docs = [docs[idx] for idx in ranked_indices if cosine_similarities[idx] > 0]

    return ranked_docs

def bm25(query, docs, k1=1.5, b=0.75):
    """
    Implements BM25 model.
    """
    # Calculate average document length for BM25
    doc_lengths = [len(doc['content'].split()) for doc in docs]
    avg_doc_length = sum(doc_lengths) / len(docs)

    query_tokens = query.lower().split()  # Tokenize the query
    scores = defaultdict(int)

    for idx, doc in enumerate(docs):
        doc_tokens = doc['content'].split()
        doc_term_freq = defaultdict(int)
        for token in doc_tokens:
            doc_term_freq[token] += 1

        for token in query_tokens:
            # Calculate BM25 score components
            idf = math.log((len(docs) - sum([1 for doc in docs if token in doc['content']]) + 0.5) / (sum([1 for doc in docs if token in doc['content']]) + 0.5) + 1.0)
            tf = doc_term_freq[token] if token in doc_term_freq else 0
            length_factor = (doc_lengths[idx] / avg_doc_length)
            score = idf * (tf * (k1 + 1)) / (tf + k1 * (1 - b + b * length_factor))
            scores[idx] += score

    # Rank documents based on BM25 scores
    ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [docs[i] for i, _ in ranked_docs if _ > 0]

## Μετρικές Αξιολόγησης
Η αξιολόγηση γίνεται μέσω μετρικών **Precision**, **Recall**, **F1 Score** και **MAP**.

In [20]:
def evaluate_system(documents, results, query):
    """
    Evaluate system using Precision, Recall, F1-Score, and MAP.
    Dynamically limits the number of relevant documents to half the total retrieved documents.
    """
    # Identify relevant documents based on the query
    query_tokens = set(query.lower().split())
    true_relevance = [1 if any(token in doc['content'] for token in query_tokens) else 0 for doc in documents]
    retrieved_relevance = [1 if doc in results else 0 for doc in documents]

    # Limit relevant documents to a specific number (e.g., half of retrieved)
    max_relevant = len(results) // 2
    true_relevance = [1 if idx < max_relevant else 0 for idx, val in enumerate(true_relevance)]

    # Calculate evaluation metrics
    precision = precision_score(true_relevance, retrieved_relevance, zero_division=0)
    recall = recall_score(true_relevance, retrieved_relevance, zero_division=0)
    f1 = f1_score(true_relevance, retrieved_relevance, zero_division=0)

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")

    def mean_average_precision(true_relevance, retrieved_relevance):
        relevant = 0
        avg_precision = 0
        for i, val in enumerate(retrieved_relevance):
            if val == 1 and true_relevance[i] == 1:
                relevant += 1
                avg_precision += relevant / (i + 1)
        return avg_precision / sum(true_relevance) if sum(true_relevance) > 0 else 0

    map_score = mean_average_precision(true_relevance, retrieved_relevance)
    print(f"MAP: {map_score:.2f}")

## Εμφάνιση Αποτελεσμάτων
Η παρουσίαση των αποτελεσμάτων αναζήτησης στον χρήστη.

In [23]:
def display_search_results(results):
    """
    Outputs search results in a user-friendly format.
    """
    if results:
        for idx, result in enumerate(results, 1):
            print(f"{idx}. Title: {result['title']}")
            print(f"Preview: {' '.join(result['content'].split()[:30])}...\n")
    else:
        print("No relevant documents found.")

## Εκτέλεση Αναζητήσεων
Η συνάρτηση `execute_search` επιτρέπει στον χρήστη να υποβάλει αιτήματα αναζήτησης, να επιλέξει το μοντέλο αναζήτησης και να δει τα αποτελέσματα.

In [ ]:
def execute_search():
    """
    Prompts user to perform dynamic searches.
    """
    while True:
        # Prompt user for query input
        query = input("Enter your search query (or type 'exit' to quit): ").strip()
        if query.lower() == 'exit':
            print("Exiting search engine. Goodbye!")
            break

        try:
            # Prompt user for the number of articles to fetch
            article_count = int(input("Enter number of articles to fetch: ").strip())
        except ValueError:
            print("Please enter a valid number for the article count.")
            continue

        # Fetch documents from Wikipedia
        documents = fetch_wikipedia_articles(query=query, article_count=article_count)

        if not documents:
            print("No articles found. Try a different query.")
            continue

        if len(documents) < article_count:
            print(f"Warning: Only {len(documents)} articles fetched instead of {article_count}.")

        print(f"Fetched {len(documents)} documents.")

        # Create inverted index for the fetched documents
        inverted_index = create_inverted_index(documents)

        # Prompt user to choose a retrieval model
        retrieval_method = input("Choose retrieval model (BOOLEAN, VSM, BM25): ").upper()
        if retrieval_method == 'BOOLEAN':
            results = boolean_search(query, documents, inverted_index)
        elif retrieval_method == 'VSM':
            results = vsm_search(query, documents)
        elif retrieval_method == 'BM25':
            results = bm25(query, documents)
        else:
            print("Invalid choice. Please select BOOLEAN, VSM, or BM25.")
            continue

        # Display the search results to the user
        display_search_results(results)

        # Evaluate system performance based on results
        evaluate_system(documents, results, query)

# Main Execution
if __name__ == "__main__":
    execute_search()

Enter your search query (or type 'exit' to quit):  sun NOT space
Enter number of articles to fetch:  15


Articles saved to 'sun_NOT_space.json'.
Fetched 15 documents.


Choose retrieval model (BOOLEAN, VSM, BM25):  BOOLEAN


1. Title: The House of the Rising Sun
Preview: house rising sun traditional folk song sometimes called rising sun blues tells person life gone wrong city new orleans many versions also urge sibling parents children avoid fate successful commercial...

2. Title: Elvis Presley
Preview: elvis aaron presley january 8 1935 august 16 1977 known mononymously elvis american singer actor known king rock roll regarded one significant cultural figures 20th century presley energized performances interpretations...

3. Title: Sun Tzu
Preview: sun tzu chinese military general strategist philosopher writer lived eastern zhou period bc sun tzu traditionally credited author art war influential work military strategy affected western east asian philosophy military...

4. Title: Ra
Preview: ra ancient egyptian rꜥ also transliterated rꜥw ancient egyptian pronunciation ˈɾiːʕuw cuneiform 𒊑𒀀 phoenician 𐤓𐤏 romanized rʿ coptic ⲣⲏ romanized rē ancient egyptian deity sun fifth dynasty 25th 24th centuries...

5. 

Enter your search query (or type 'exit' to quit):  car OR BMW 
Enter number of articles to fetch:  15


Articles saved to 'car_OR_BMW.json'.
Fetched 15 documents.


Choose retrieval model (BOOLEAN, VSM, BM25):  BOOLEAN


1. Title: Mini (marque)
Preview: mini stylised mini british automotive brand founded oxford 1969 owned german multinational automotive company bmw since 2000 used range small cars assembled united kingdom austria netherlands 16 february 2024 germany...

2. Title: BMW 3 Series
Preview: bmw 3 series line compact executive cars manufactured german automaker bmw since may 1975 successor 02 series produced seven generations first generation 3 series available saloon model range expanded include...

3. Title: D-segment
Preview: 4th category european segments passenger cars described large cars equivalent euro ncap large family car size class definition car category used north america compact executive cars part size category sales...

4. Title: BMW 7 Series
Preview: bmw 7 series luxury sedan manufactured marketed german automaker bmw since 1977 successor bmw e3 new six sedan seventh generation 7 series bmw flagship car available sedan bodystyle including long...

5. Title: BMW M4
Preview: bm